# GLASS Demo

In [ ]:
import urllib.request

from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

from glass.inference.glass_runner import GlassRunner
from glass.utils.logger import setup_logger
from glass.utils.visualizer import visualize

setup_logger()

In [ ]:
#@ Installing Detectron2 and PyYaml which are prerequisite
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!python -m pip install -U PyYAML

#@ Install GLASS
!git clone 'https://github.com/amazon-science/glass-text-spotting'
%cd glass-text-spotting
!python -m pip install .

In [ ]:
#@ Download the pretrained TextOCR model
!mkdir assets
!wget 'https://glass-text-spotting.s3.eu-west-1.amazonaws.com/models/glass_250k_full_textocr_finetune.pth' -O 'assets/glass_textocr.pth'

In [ ]:
# Setting up the runner for running inference. The runner also includes the text encoder
model_path = 'assets/glass_textocr.pth'
config_path = '/hiero_efs/HieroUsers/tsiper/HieroDeploy/TextractGlassOCR/configs/glass_finetune_textocr.yaml'
glass_runner = GlassRunner(model_path=model_path, config_path=config_path, post_process=True)


In [ ]:
#@ Choose an image
image_id = 3  #@param {type:"slider", min:1, max:4, step:1}
image_url = f'https://raw.githubusercontent.com/Yuliang-Liu/Curve-Text-Detector/master/images/demo/{image_id}.jpg'

#@markdown ---
#@markdown Or provide a url path to cropped text image (Optional).
#@markdown ### Enter a file path:

file_path = ""  #@param {type:"string"}
if file_path:
    image_url = file_path

In [ ]:
# Obtaining the image:
file_name, headers = urllib.request.urlretrieve(image_url)
image = np.asarray(Image.open(file_name).convert('RGB')) # Inference is not supported for images with alpha channel
plt.imshow(image)
plt.show()

In [ ]:
# Running GLASS on the input image
preds = glass_runner(image)

In [ ]:
# Visualizing the results, hover on the detections for viewing the confidence
figure = visualize(preds=preds, image=image, text_encoder=glass_runner.text_encoder, vis_width=720, vis_text=True)
figure.show()